In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH17-LangGraph-Modules")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH17-LangGraph-Modules


In [2]:
from typing import Annotated
from typing_extensions import TypedDict

from langchain_teddynote.tools.tavily import TavilySearch
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_teddynote.graphs import visualize_graph

# 1. 상태 정의
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 2. 도구 정의 및 바인딩
tool = TavilySearch(max_results=3)

tools = [tool]

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools(tools)

# 3. 노드 추가
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 상태 그래프 생성
graph_builder = StateGraph(State)

# 챗봇 노드 추가
graph_builder.add_node("chatbot", chatbot)

# 도구 노드 생성 및 추가
tool_node = ToolNode(tools=tools)

graph_builder.add_node("tools", tool_node)

# 조건부 엣지
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# 4. 엣지 추가
graph_builder.add_edge("tools", "chatbot")

graph_builder.add_edge(START, "chatbot")

graph_builder.add_edge("chatbot", END)

# 5. 그래프 컴파일
memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

In [4]:
from langchain_core.runnables import RunnableConfig

question = "LangGraph가 무엇인지 조사하여 알려주세요."

input = State(messages=[("user", question)])

config = RunnableConfig(
    configurable={"thread_id": "1"}
)

In [5]:
list(graph.channels)

['messages',
 '__start__',
 'chatbot',
 'branch:to:chatbot',
 'tools',
 'branch:to:tools',
 'start:chatbot']

In [6]:
# 그래프 스트림 호출
events = graph.stream(
    input=input,
    config=config,
    interrupt_before=["tools"],
    stream_mode="values"
)

# 이벤트 반복 처리
for event in events:
    # 메시지가 이벤트에 포함된 경우
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

LangGraph가 무엇인지 조사하여 알려주세요.
================================== Ai Message ==================================
Tool Calls:
  tavily_web_search (call_vcpChqgKIV79kmIHhYcsdPlH)
 Call ID: call_vcpChqgKIV79kmIHhYcsdPlH
  Args:
    query: LangGraph


In [8]:
# 그래프 상태 스냅샷 생성
snapshot = graph.get_state(config)

# 가장 최근 메시지 추출
last_message = snapshot.values["messages"][-1]

# 메시지 출력
last_message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  tavily_web_search (call_vcpChqgKIV79kmIHhYcsdPlH)
 Call ID: call_vcpChqgKIV79kmIHhYcsdPlH
  Args:
    query: LangGraph


# 사람의 개입 (Human-in-the-loop)

In [9]:
modified_search_result = """[수정된 웹 검색 결과]
LangGraph는 상태 기반의 다중 액터 애플리케이션을 LLM을 활용해 구축할 수 있도록 지원합니다.
LangGraph는 사이클 흐름, 제어 가능성, 지속성, 클라우드 배포 기능을 제공하는 오픈 소스 라이브러리입니다.

자세한 튜토리얼은 [LangGraph 튜토리얼](https://langchain-ai.github.io/langgraph/tutorials/) 과
테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요."""

print(modified_search_result)

[수정된 웹 검색 결과]
LangGraph는 상태 기반의 다중 액터 애플리케이션을 LLM을 활용해 구축할 수 있도록 지원합니다.
LangGraph는 사이클 흐름, 제어 가능성, 지속성, 클라우드 배포 기능을 제공하는 오픈 소스 라이브러리입니다.

자세한 튜토리얼은 [LangGraph 튜토리얼](https://langchain-ai.github.io/langgraph/tutorials/) 과
테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요.


In [11]:
last_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vcpChqgKIV79kmIHhYcsdPlH', 'function': {'arguments': '{"query":"LangGraph"}', 'name': 'tavily_web_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 101, 'total_tokens': 120, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BPP7JBOMKYnXWEKmes4HI2w4pYB9M', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-af11414d-08c5-482f-8d61-5b22e861c268-0', tool_calls=[{'name': 'tavily_web_search', 'args': {'query': 'LangGraph'}, 'id': 'call_vcpChqgKIV79kmIHhYcsdPlH', 'type': 'tool_call'}], usage_metadata={'input_tokens': 101, 'output_tokens': 19, 'total_tokens': 120, 'input_token_details': {'audio': 

In [12]:
# 수정하고자 하는 'ToolMessage'의 'tool_call_id' 추출
tool_call_id = last_message.tool_calls[0]["id"]
print(tool_call_id)

call_vcpChqgKIV79kmIHhYcsdPlH


In [13]:
from langchain_core.messages import AIMessage, ToolMessage

new_messages = [
    # LLM API의 도구 호출과 일치하는 ToolMessage 필요
    ToolMessage(
        content=modified_search_result,
        tool_call_id=tool_call_id,
    )
]

new_messages[-1].pretty_print()

================================= Tool Message =================================

[수정된 웹 검색 결과]
LangGraph는 상태 기반의 다중 액터 애플리케이션을 LLM을 활용해 구축할 수 있도록 지원합니다.
LangGraph는 사이클 흐름, 제어 가능성, 지속성, 클라우드 배포 기능을 제공하는 오픈 소스 라이브러리입니다.

자세한 튜토리얼은 [LangGraph 튜토리얼](https://langchain-ai.github.io/langgraph/tutorials/) 과
테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요.


In [14]:
graph.update_state(
    # 업데이트할 상태 지정
    config,
    # 제공할 업데이트된 값. `State`의 메시지는 "추가 전용"으로 기존 상태에 추가됨
    {"messages": new_messages},
    as_node="tools",
)

print("(최근 1개의 메시지 출력)\n")
print(graph.get_state(config).values["messages"][-1])

(최근 1개의 메시지 출력)

content='[수정된 웹 검색 결과]\nLangGraph는 상태 기반의 다중 액터 애플리케이션을 LLM을 활용해 구축할 수 있도록 지원합니다.\nLangGraph는 사이클 흐름, 제어 가능성, 지속성, 클라우드 배포 기능을 제공하는 오픈 소스 라이브러리입니다.\n\n자세한 튜토리얼은 [LangGraph 튜토리얼](https://langchain-ai.github.io/langgraph/tutorials/) 과\n테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요.' id='032cb8fb-c95b-43d7-845e-53aceb5985a8' tool_call_id='call_vcpChqgKIV79kmIHhYcsdPlH'


In [15]:
snapshot = graph.get_state(config)
snapshot.next

('chatbot',)

In [16]:
# `None`는 현재 상태에 아무것도 추가하지 않음
events = graph.stream(None, config, stream_mode="values")

# 이벤트 반복 처리
for event in events:
    # 메시지가 이벤트에 포함된 경우
    if "messages" in event:
        # 마지막 메시지의 예쁜 출력
        event["messages"][-1].pretty_print()

================================= Tool Message =================================

[수정된 웹 검색 결과]
LangGraph는 상태 기반의 다중 액터 애플리케이션을 LLM을 활용해 구축할 수 있도록 지원합니다.
LangGraph는 사이클 흐름, 제어 가능성, 지속성, 클라우드 배포 기능을 제공하는 오픈 소스 라이브러리입니다.

자세한 튜토리얼은 [LangGraph 튜토리얼](https://langchain-ai.github.io/langgraph/tutorials/) 과
테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요.
================================== Ai Message ==================================

LangGraph는 상태 기반의 다중 액터 애플리케이션을 대형 언어 모델(LLM)을 활용하여 구축할 수 있도록 지원하는 오픈 소스 라이브러리입니다. 이 라이브러리는 사이클 흐름, 제어 가능성, 지속성, 클라우드 배포 기능 등을 제공하여 개발자들이 더욱 효율적으로 애플리케이션을 설계하고 배포할 수 있게 해줍니다.

자세한 튜토리얼은 다음 링크에서 확인할 수 있습니다:
- [LangGraph 튜토리얼](https://langchain-ai.github.io/langgraph/tutorials/)
- [랭체인 한국어 튜토리얼](https://wikidocs.net/233785)


In [17]:
# 그래프 상태 스냅샷 생성
snapshot = graph.get_state(config)

# 최근 세 개의 메시지 출력
for message in snapshot.values["messages"]:
    message.pretty_print()

================================ Human Message =================================

LangGraph가 무엇인지 조사하여 알려주세요.
================================== Ai Message ==================================
Tool Calls:
  tavily_web_search (call_vcpChqgKIV79kmIHhYcsdPlH)
 Call ID: call_vcpChqgKIV79kmIHhYcsdPlH
  Args:
    query: LangGraph
================================= Tool Message =================================

[수정된 웹 검색 결과]
LangGraph는 상태 기반의 다중 액터 애플리케이션을 LLM을 활용해 구축할 수 있도록 지원합니다.
LangGraph는 사이클 흐름, 제어 가능성, 지속성, 클라우드 배포 기능을 제공하는 오픈 소스 라이브러리입니다.

자세한 튜토리얼은 [LangGraph 튜토리얼](https://langchain-ai.github.io/langgraph/tutorials/) 과
테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요.
================================== Ai Message ==================================

LangGraph는 상태 기반의 다중 액터 애플리케이션을 대형 언어 모델(LLM)을 활용하여 구축할 수 있도록 지원하는 오픈 소스 라이브러리입니다. 이 라이브러리는 사이클 흐름, 제어 가능성, 지속성, 클라우드 배포 기능 등을 제공하여 개발자들이 더욱 효율적으로 애플리케이션을 설계하고 배포할 수 있게 해줍니다.

자세한 튜토리얼은 다음 링크에서 확인할 수 있습니다:
- [LangGraph 튜토리얼](https://

In [18]:
# 다음 상태 출력
print(snapshot.next)

()


# Interrupt 후 메시지 상태 업데이트 - 이어서 진행

In [19]:
from langchain_teddynote.graphs import generate_random_hash

thread_id = generate_random_hash()
print(f"thread_id: {thread_id}")

question = "LangGraph 에 대해서 배워보고 싶습니다. 유용한 자료를 추천해 주세요!"

# 초기 입력 상태를 정의
input = State(messages=[("user", question)])

# 새로운 config 생성
config = {"configurable": {"thread_id": thread_id}}

events = graph.stream(
    input=input,
    config=config,
    interrupt_before=["tools"],
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

thread_id: da7be0
================================ Human Message =================================

LangGraph 에 대해서 배워보고 싶습니다. 유용한 자료를 추천해 주세요!
================================== Ai Message ==================================
Tool Calls:
  tavily_web_search (call_x97g4sjFCID0y9R8W42AUelP)
 Call ID: call_x97g4sjFCID0y9R8W42AUelP
  Args:
    query: LangGraph 소개


In [20]:
# config 를 복사
config_copy = config.copy()

In [21]:
from langchain_core.messages import AIMessage

# 스냅샷 상태 가져오기
snapshot = graph.get_state(config)

# messages 의 마지막 메시지 가져오기
existing_message = snapshot.values["messages"][-1]

# 메시지 ID 출력
print("Message ID", existing_message.id)

Message ID run-d4934350-ee04-4cb8-a32f-b3c7ea24d77e-0


In [22]:
# 첫 번째 도구 호출 출력
print(existing_message.tool_calls[0])

{'name': 'tavily_web_search', 'args': {'query': 'LangGraph 소개'}, 'id': 'call_x97g4sjFCID0y9R8W42AUelP', 'type': 'tool_call'}


In [23]:
# tool_calls 를 복사하여 새로운 도구 호출 생성
new_tool_call = existing_message.tool_calls[0].copy()

# 쿼리 매개변수 업데이트(갱신)
new_tool_call["args"] = {"query": "LangGraph site:teddylee777.github.io"}
new_tool_call

{'name': 'tavily_web_search',
 'args': {'query': 'LangGraph site:teddylee777.github.io'},
 'id': 'call_x97g4sjFCID0y9R8W42AUelP',
 'type': 'tool_call'}

In [24]:
# AIMessage 생성
new_message = AIMessage(
    content=existing_message.content,
    tool_calls=[new_tool_call],
    # 중요! ID는 메시지를 상태에 추가하는 대신 교체하는 방법
    id=existing_message.id,
)

print(new_message.id)

# 수정한 메시지 출력
new_message.pretty_print()

run-d4934350-ee04-4cb8-a32f-b3c7ea24d77e-0
================================== Ai Message ==================================
Tool Calls:
  tavily_web_search (call_x97g4sjFCID0y9R8W42AUelP)
 Call ID: call_x97g4sjFCID0y9R8W42AUelP
  Args:
    query: LangGraph site:teddylee777.github.io


In [25]:
# 업데이트된 도구 호출 출력
print(new_message.tool_calls[0])

# 메시지 ID 출력
print("\nMessage ID", new_message.id)

# 상태 업데이트
graph.update_state(config, {"messages": [new_message]})

{'name': 'tavily_web_search', 'args': {'query': 'LangGraph site:teddylee777.github.io'}, 'id': 'call_x97g4sjFCID0y9R8W42AUelP', 'type': 'tool_call'}

Message ID run-d4934350-ee04-4cb8-a32f-b3c7ea24d77e-0


{'configurable': {'thread_id': 'da7be0',
  'checkpoint_ns': '',
  'checkpoint_id': '1f020194-6c05-6f66-8002-e60315b91858'}}

In [26]:
# 마지막 메시지의 도구 호출 가져오기
graph.get_state(config).values["messages"][-1].tool_calls

[{'name': 'tavily_web_search',
  'args': {'query': 'LangGraph site:teddylee777.github.io'},
  'id': 'call_x97g4sjFCID0y9R8W42AUelP',
  'type': 'tool_call'}]

In [27]:
# 그래프 스트림에서 이벤트 수신
events = graph.stream(None, config, stream_mode="values")

# 각 이벤트에 대한 처리
for event in events:
    # 메시지가 있는 경우 마지막 메시지 출력
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  tavily_web_search (call_x97g4sjFCID0y9R8W42AUelP)
 Call ID: call_x97g4sjFCID0y9R8W42AUelP
  Args:
    query: LangGraph site:teddylee777.github.io
================================= Tool Message =================================
Name: tavily_web_search

[{"url": "https://teddylee777.github.io/langgraph/langgraph-multi-agent-collaboration/", "title": "LangGraph - Multi-Agent Collaboration(다중 협업 에이전트 ...", "content": "failed", "score": 0.2976286, "raw_content": "LangGraph - Multi-Agent Collaboration(다중 협업 에이전트) 로 복잡한 테스크를 수행하는 LLM 어플리케이션 제작 - 테디노트\n\nSkip to primary navigation\nSkip to content\nSkip to footer\n\n테디노트 데이터와 인공지능을 좋아하는 개발자 노트\n\n검색\n카테고리\n태그\n연도\n강의\n어바웃미\n\n토글 메뉴\n\nHome \n/3.  Langgraph \n/5.  LangGraph - Multi-Agent Collaboration(다중 협업 에이전트) 로 복잡한 테스크를 수행하는 LLM 어플리케이션 제작\n\n🔥알림🔥\n① 테디노트 유튜브 - 구경하러 가기!\n② LangChain 한국어 튜토리얼 바로가기 👀\n③ 랭체인 노트 무료 전자책(wikidocs) 바로가기 🙌\n④ RAG 비법노트 Lang

In [28]:
# 이벤트 스트림 생성
events = graph.stream(
    {
        "messages": (
            "user",
            "내가 지금까지 배운 내용에 대해서 매우 친절하고 정성스럽게 한국어로 답변해줘! 출처를 반드시 포함해줘!",
        )
    },
    config,
    stream_mode="values",
)

# 메시지 이벤트 처리
for event in events:
    if "messages" in event:
        # 마지막 메시지 출력
        event["messages"][-1].pretty_print()

================================ Human Message =================================

내가 지금까지 배운 내용에 대해서 매우 친절하고 정성스럽게 한국어로 답변해줘! 출처를 반드시 포함해줘!
================================== Ai Message ==================================

물론입니다! 지금까지 배운 LangGraph에 대한 내용을 정리해 드리겠습니다.

### LangGraph란?
LangGraph는 다중 에이전트 협업을 통해 복잡한 문제를 해결하는 데 도움을 주는 Python 라이브러리입니다. 이 라이브러리는 다양한 에이전트가 각기 다른 도메인에서 전문화된 지식을 활용하여 공동으로 작업을 수행하도록 돕습니다. LangGraph는 특히 AI와 언어 모델(Large Language Models, LLMs)을 활용하여 문제 해결에 접근할 수 있는 강력한 도구입니다.

### 주요 개념
1. **다중 에이전트 협업**: 여러 AI 에이전트가 협력하여 각자의 전문 영역에서 정보를 공유하고 문제를 해결합니다. 이를 통해 복잡한 작업을 보다 효과적으로 처리할 수 있습니다.
   
2. **도구 정의**: 각 에이전트는 특정 작업을 수행할 수 있는 도구를 정의하고 사용할 수 있습니다. 예를 들어, 문서 검색 도구나 데이터 처리 도구 등이 있습니다.

3. **그래프 생성**: LangGraph는 에이전트들이 서로 통신하고 협력하는 그래프 구조를 생성합니다. 이를 통해 각 에이전트의 상태를 관리하고 작업 흐름을 제어할 수 있습니다.

### 사용 예시
- 예를 들어, 한 에이전트가 데이터를 수집하고, 다른 에이전트는 그 데이터를 분석하여 최종 결과를 도출하는 방식으로 협업할 수 있습니다. 이러한 방식은 "나누고 정복하기(divide-and-conquer)" 접근법에 기반합니다.

### 관련 자료
1. **[LangGraph - Multi-Agent C

In [29]:
graph.get_state(config).values["messages"][-1].pretty_print()

================================== Ai Message ==================================

물론입니다! 지금까지 배운 LangGraph에 대한 내용을 정리해 드리겠습니다.

### LangGraph란?
LangGraph는 다중 에이전트 협업을 통해 복잡한 문제를 해결하는 데 도움을 주는 Python 라이브러리입니다. 이 라이브러리는 다양한 에이전트가 각기 다른 도메인에서 전문화된 지식을 활용하여 공동으로 작업을 수행하도록 돕습니다. LangGraph는 특히 AI와 언어 모델(Large Language Models, LLMs)을 활용하여 문제 해결에 접근할 수 있는 강력한 도구입니다.

### 주요 개념
1. **다중 에이전트 협업**: 여러 AI 에이전트가 협력하여 각자의 전문 영역에서 정보를 공유하고 문제를 해결합니다. 이를 통해 복잡한 작업을 보다 효과적으로 처리할 수 있습니다.
   
2. **도구 정의**: 각 에이전트는 특정 작업을 수행할 수 있는 도구를 정의하고 사용할 수 있습니다. 예를 들어, 문서 검색 도구나 데이터 처리 도구 등이 있습니다.

3. **그래프 생성**: LangGraph는 에이전트들이 서로 통신하고 협력하는 그래프 구조를 생성합니다. 이를 통해 각 에이전트의 상태를 관리하고 작업 흐름을 제어할 수 있습니다.

### 사용 예시
- 예를 들어, 한 에이전트가 데이터를 수집하고, 다른 에이전트는 그 데이터를 분석하여 최종 결과를 도출하는 방식으로 협업할 수 있습니다. 이러한 방식은 "나누고 정복하기(divide-and-conquer)" 접근법에 기반합니다.

### 관련 자료
1. **[LangGraph - Multi-Agent Collaboration](https://teddylee777.github.io/langgraph/langgraph-multi-agent-collaboration/)**: 이 자료에서는 다중 에이전트 협업을 통해 복잡한 문제를 해결하는 방법을 설명합니다

# 지난 스냅샷의 결과 수정 및 Replay


In [30]:
to_replay_state = None

# 상태 기록 가져오기
for state in graph.get_state_history(config):

    messages = state.values["messages"]

    if len(messages) > 0:
        print(state.values["messages"][-1].id)
        # 메시지 수 및 다음 상태 출력
        print("메시지 수: ", len(state.values["messages"]), "다음 노드: ", state.next)
        print("-" * 80)
        # 특정 상태 선택 기준: 채팅 메시지 수
        if len(state.values["messages"]) == 2:
            # 특정 메시지 ID 선택
            to_replay_state = state

run-b6274d83-0f6f-4b6d-9b51-c2290625edcc-0
메시지 수:  6 다음 노드:  ()
--------------------------------------------------------------------------------
2401e4ac-e06d-4f4b-aa04-33af1aa24c83
메시지 수:  5 다음 노드:  ('chatbot',)
--------------------------------------------------------------------------------
run-4b4df3d1-af67-4085-ab06-140d419bc46d-0
메시지 수:  4 다음 노드:  ('__start__',)
--------------------------------------------------------------------------------
run-4b4df3d1-af67-4085-ab06-140d419bc46d-0
메시지 수:  4 다음 노드:  ()
--------------------------------------------------------------------------------
b9a973b1-88a8-4740-8959-5b24c54725e0
메시지 수:  3 다음 노드:  ('chatbot',)
--------------------------------------------------------------------------------
run-d4934350-ee04-4cb8-a32f-b3c7ea24d77e-0
메시지 수:  2 다음 노드:  ('tools',)
--------------------------------------------------------------------------------
run-d4934350-ee04-4cb8-a32f-b3c7ea24d77e-0
메시지 수:  2 다음 노드:  ('tools',)
------------------------------

In [31]:
from langchain_teddynote.messages import display_message_tree

# 선택한 메시지 가져오기
existing_message = to_replay_state.values["messages"][-1]

# 메시지 트리 출력
display_message_tree(existing_message)

    content: ""
    additional_kwargs:
        tool_calls:
            index [0]
                id: "call_x97g4sjFCID0y9R8W42AUelP"
                function: {"arguments": "{"query":"LangGraph 소개"}", "name": "tavily_web_search"}
                type: "function"
        refusal: None
    response_metadata:
        token_usage:
            completion_tokens: 20
            prompt_tokens: 110
            total_tokens: 130
            completion_tokens_details: {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}
            prompt_tokens_details: {"audio_tokens": 0, "cached_tokens": 0}
        model_name: "gpt-4o-mini-2024-07-18"
        system_fingerprint: "fp_dbaca60df0"
        id: "chatcmpl-BPPImqhvE57TjF6ytKZ0lohltol0F"
        finish_reason: "tool_calls"
        logprobs: None
    type: "ai"
    name: None
    id: "run-d4934350-ee04-4cb8-a32f-b3c7ea24d77e-0"
    example: False
    tool_calls:
        index [0]
            name

In [32]:
tool_call = existing_message.tool_calls[0].copy()
tool_call["args"] = {"query": "LangGraph human-in-the-loop workflow site:reddit.com"}
tool_call

{'name': 'tavily_web_search',
 'args': {'query': 'LangGraph human-in-the-loop workflow site:reddit.com'},
 'id': 'call_x97g4sjFCID0y9R8W42AUelP',
 'type': 'tool_call'}

In [33]:
# AIMessage 생성
new_message = AIMessage(
    content=existing_message.content,
    tool_calls=[tool_call],
    # 중요! ID는 메시지를 상태에 추가하는 대신 교체하는 방법
    id=existing_message.id,
)

# 수정한 메시지 출력
new_message.tool_calls[0]["args"]

{'query': 'LangGraph human-in-the-loop workflow site:reddit.com'}

In [34]:
# 업데이트 전 메시지 확인
graph.get_state(to_replay_state.config).values["messages"][-1].tool_calls

[{'name': 'tavily_web_search',
  'args': {'query': 'LangGraph 소개'},
  'id': 'call_x97g4sjFCID0y9R8W42AUelP',
  'type': 'tool_call'}]

In [35]:
# 상태 업데이트
updated_state = graph.update_state(
    to_replay_state.config,
    {"messages": [new_message]},
)
updated_state

{'configurable': {'thread_id': 'da7be0',
  'checkpoint_ns': '',
  'checkpoint_id': '1f02019c-619d-6387-8002-1e2c8ae3c363'}}

In [36]:
# config 에는 updated_state 를 전달합니다. 이는 임의로 갱신한 상태를 전달하는 것입니다.
for event in graph.stream(None, updated_state, stream_mode="values"):
    # 메시지가 이벤트에 포함된 경우
    if "messages" in event:
        # 마지막 메시지 출력
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  tavily_web_search (call_x97g4sjFCID0y9R8W42AUelP)
 Call ID: call_x97g4sjFCID0y9R8W42AUelP
  Args:
    query: LangGraph human-in-the-loop workflow site:reddit.com
================================= Tool Message =================================
Name: tavily_web_search

[{"title": "Human intervention in agent workflows : r/LangChain - Reddit", "url": "https://www.reddit.com/r/LangChain/comments/1bjnmu4/human_intervention_in_agent_workflows/", "content": "Reddit - Dive into anything Open menu Open navigation  Go to Reddit Home Go to LangChain r/LangChain r/LangChain When building LLM workflows with LangChain/LangGraph what's the best way to build a node in the workflow where a human can validate/approve/reject a flow? Top Posts Reddit Reddit Reddit Action Movies & Series Animated Movies & Series Comedy Movies & Series Crime, Mystery, & Thriller Movies & Series Documentary Movies & Series Drama Mo

In [37]:
# 최종 결과 출력
for msg in graph.get_state(config).values["messages"]:
    msg.pretty_print()

================================ Human Message =================================

LangGraph 에 대해서 배워보고 싶습니다. 유용한 자료를 추천해 주세요!
================================== Ai Message ==================================
Tool Calls:
  tavily_web_search (call_x97g4sjFCID0y9R8W42AUelP)
 Call ID: call_x97g4sjFCID0y9R8W42AUelP
  Args:
    query: LangGraph human-in-the-loop workflow site:reddit.com
================================= Tool Message =================================
Name: tavily_web_search

[{"title": "Human intervention in agent workflows : r/LangChain - Reddit", "url": "https://www.reddit.com/r/LangChain/comments/1bjnmu4/human_intervention_in_agent_workflows/", "content": "Reddit - Dive into anything Open menu Open navigation  Go to Reddit Home Go to LangChain r/LangChain r/LangChain When building LLM workflows with LangChain/LangGraph what's the best way to build a node in the workflow where a human can validate/approve/reject a flow? Top Posts Reddit Reddit Reddit Action Movies & Series A